## Jupyterhub on Kubernetes

This notebook can be used to launch a standard instance of Jupyterhub which includes a proxy server, and an instance of Jupyterhub. 

Notes on the Server: 
`/etc/jupyterhub/config` is where the configuration is for jupyterhub. 



In [ ]:
#This will import some required libraries.
import sys 
import ruamel.yaml 
#This is your configuration file. 
general_yaml='../../config/config.yaml'
with open(general_yaml, 'r') as yaml:
    cf=ruamel.yaml.round_trip_load(yaml, preserve_quotes=True)

jupyterhub_yaml='../../config/pachyderm/config.yaml'
with open(jupyterhub_yaml, 'r') as j_yaml:
    cf_p=ruamel.yaml.round_trip_load(j_yaml, preserve_quotes=True)

    #This will allow us to import some useful utilities. 
if cf['docker']:
    cf_p['path']=cf['docker_path']
else:
    cf_p['path']=cf['local_path']

sys.path.append(cf_p['path']+"/lib/kuberutils")   
import importlib
import kuberutils as ku
importlib.reload(ku)
#This will load common commands for your cluster
cf_p=ku.pachyderm_commands(cf_p)
print(ruamel.yaml.dump(cf_p, sys.stdout, Dumper=ruamel.yaml.RoundTripDumper))

#helm install jupyterhub/jupyterhub \
#    --version=v0.4 \
#    --name=<YOUR-RELEASE-NAME> \
#    --namespace=<YOUR-NAMESPACE> \
#    -f config.yaml

In [ ]:
#For some reason python function doesn't work.   
#Need Service 
!az login

In [ ]:
print(ku.bash_command('install_pachyderm',cf_p))

In [ ]:
print(ku.bash_command('clone',cf_p))

In [ ]:
cf_p['storage_key']=(ku.bash_command('get_storage_key',cf_p))
print(cf_p['storage_key'])

In [ ]:
#create_blank_vhd STORAGE_ACCOUNT_NAME STORAGE_ACCOUNT_KEY CONTAINER_NAME VHD_NAME [VHD_SIZE] [--verbose]
cf_p['microsoft_vhd']="go run /home/jovyan/work/pachyderm/etc/microsoft/create-blank-vhd/create_blank_vhd.go "+cf_p['storage_account']+" '"+cf_p['storage_key']+"' "+cf_p['container_name']+" "+cf_p['storage_name']+" "+str(cf_p['storage_size'])+"G"
cf_p['ms_blob']= "az storage blob list --container="+cf_p['container_name']+ " --account-name="+cf_p['storage_account']+" --account-key='"+cf_p['storage_key']+"'"


In [ ]:
cf_p['microsoft_vhd']

In [ ]:
cf_p['volume_uri']=ku.bash_command('microsoft_vhd',cf_p)
print(cf_p['volume_uri'])

In [ ]:
cf_p['volume_uri']

In [ ]:
cf_p['ms_deploy']= "pachctl deploy microsoft "+ cf_p['container_name'] +" "+ cf_p['storage_account'] +" '"+ cf_p['storage_key']+"' "+ str(cf_p['storage_size']) + " --static-etcd-volume="+cf_p['volume_uri']
cf_p['ms_deploy']

In [ ]:
ku.bash_command('ms_deploy',cf_p)

In [ ]:
print(ku.bash_command('kubectl get all'))

In [ ]:
print(ku.bash_command('pachctl port-forward &'))

In [ ]:
!pachctl version

In [ ]:
print(ku.bash_command("az storage account list | jq '.[].name'"))

In [ ]:
print(ku.bash_command('blob',cf_p))

In [ ]:
!pachctl create-repo images